Preparacion del entrono de trabajo

# Experimento 1 Prepacion del entorno

In [ ]:
# Instalar Apache Spark y Hadoop
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Verificar que el archivo se descargó
!ls -lh




total 384M
-rw-r--r--  1 root root  584 Oct 23 16:08 analysis.py
-rw-r--r--  1 root root 1.3M Oct 23 16:09 Films_2.xlsx
drwxr-xr-x  1 root root 4.0K Oct 21 13:22 sample_data
drwxr-xr-x 13 1000 1000 4.0K Sep  9  2023 spark-3.5.0-bin-hadoop3
-rw-r--r--  1 root root 382M Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
import os
import findspark

# Configurar las rutas de Spark y Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Iniciar Spark
findspark.init()

In [ ]:
## Iniciar la sesión de Spark
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("Prueba_Spark").getOrCreate()

# Verificar que Spark esté activo
print("Versión de Spark:", spark.version)

Versión de Spark: 3.5.0


In [ ]:

## DataFrame de prueba
from pyspark.sql import Row

# Crear un DataFrame de ejemplo con datos simples
data = [Row(nombre="Juan", edad=28),
        Row(nombre="Ana", edad=24),
        Row(nombre="Pedro", edad=30)]

# Convertir la lista de datos a un DataFrame de Spark
df = spark.createDataFrame(data)

# Mostrar el contenido del DataFrame
df.show()

# Hacer una operación básica: contar el número de filas
print(f"Total de filas en el DataFrame de prueba: {df.count()}")

+------+----+
|nombre|edad|
+------+----+
|  Juan|  28|
|   Ana|  24|
| Pedro|  30|
+------+----+

Total de filas en el DataFrame de prueba: 3


In [ ]:
## Operaciones adicionales de prueba

# Filtrar los datos donde la edad es mayor que 25
df.filter(df.edad > 25).show()

# Agrupar por un valor (en este caso no tiene sentido porque son pocos datos, pero sirve de prueba)
df.groupBy("nombre").count().show()

+------+----+
|nombre|edad|
+------+----+
|  Juan|  28|
| Pedro|  30|
+------+----+

+------+-----+
|nombre|count|
+------+-----+
|  Juan|    1|
| Pedro|    1|
|   Ana|    1|
+------+-----+



In [ ]:
!pip install -q openpyxl  # Para manejar archivos Excel
!pip install -q pandas  # pandas también es necesario

In [ ]:
# Iniciar Spark
findspark.init()

# Crear una sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("Excel_Spark").getOrCreate()

# Verificar que Spark esté funcionando correctamente
spark

# Experiento 2 importando xls films

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

# Crear SparkSession (en lugar de SQLContext)
spark = SparkSession.builder.master("local[*]").appName("ExcelToSpark").getOrCreate()

# Leer el archivo Excel con pandas (todas las hojas)
excel_file = "/content/Films_2.xlsx"  # Cambia esto por el nombre de tu archivo subido
sheets_dict = pd.read_excel(excel_file, sheet_name=None)  # Cargar todas las hojas

# Verificar que las hojas se hayan cargado correctamente
print(f"Hojas cargadas: {sheets_dict.keys()}")

# Convertir cada hoja de pandas a DataFrame de Spark y guardarlo en un diccionario
spark_dfs = {}
for sheet_name, df in sheets_dict.items():
    if df.shape[0] > 0:  # Verificar si la hoja no está vacía
        spark_dfs[sheet_name] = spark.createDataFrame(df)
        print(f"Hoja '{sheet_name}' convertida a DataFrame de Spark con {df.shape[0]} filas.")
    else:
        print(f"La hoja '{sheet_name}' está vacía y no se convertirá.")

# Mostrar los nombres de las hojas convertidas y el número de filas en cada DataFrame de Spark
for sheet_name, df in spark_dfs.items():
    print(f"Hoja: {sheet_name}, Número de filas: {df.count()}")

Hojas cargadas: dict_keys(['MER', 'film', 'inventory', 'rental', 'customer', 'store'])
La hoja 'MER' está vacía y no se convertirá.
Hoja 'film' convertida a DataFrame de Spark con 1003 filas.
Hoja 'inventory' convertida a DataFrame de Spark con 4581 filas.
Hoja 'rental' convertida a DataFrame de Spark con 16044 filas.
Hoja 'customer' convertida a DataFrame de Spark con 1392 filas.
Hoja 'store' convertida a DataFrame de Spark con 2 filas.
Hoja: film, Número de filas: 1003
Hoja: inventory, Número de filas: 4581
Hoja: rental, Número de filas: 16044
Hoja: customer, Número de filas: 1392
Hoja: store, Número de filas: 2


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

# Crear SparkSession
spark = SparkSession.builder.master("local[*]").appName("ExcelToSpark").getOrCreate()

# Leer el archivo Excel con pandas (todas las hojas)
excel_file = "/content/Films_2.xlsx"  # Cambia esto por el nombre de tu archivo subido
sheets_dict = pd.read_excel(excel_file, sheet_name=None)  # Cargar todas las hojas

# Verificar que las hojas se hayan cargado correctamente
print(f"Hojas cargadas: {sheets_dict.keys()}")

# Convertir cada hoja de pandas a DataFrame de Spark y guardarlo en un diccionario
spark_dfs = {}
for sheet_name, df in sheets_dict.items():
    if df.shape[0] > 0:  # Verificar si la hoja no está vacía
        spark_dfs[sheet_name] = spark.createDataFrame(df)
        print(f"Hoja '{sheet_name}' convertida a DataFrame de Spark con {df.shape[0]} filas.")
    else:
        print(f"La hoja '{sheet_name}' está vacía y no se convertirá.")

# Mostrar los nombres de las hojas convertidas y el número de filas en cada DataFrame de Spark
for sheet_name, df in spark_dfs.items():
    print(f"Hoja: {sheet_name}, Número de filas: {df.count()}")

Hojas cargadas: dict_keys(['MER', 'film', 'inventory', 'rental', 'customer', 'store'])
La hoja 'MER' está vacía y no se convertirá.
Hoja 'film' convertida a DataFrame de Spark con 1003 filas.
Hoja 'inventory' convertida a DataFrame de Spark con 4581 filas.
Hoja 'rental' convertida a DataFrame de Spark con 16044 filas.
Hoja 'customer' convertida a DataFrame de Spark con 1392 filas.
Hoja 'store' convertida a DataFrame de Spark con 2 filas.
Hoja: film, Número de filas: 1003
Hoja: inventory, Número de filas: 4581
Hoja: rental, Número de filas: 16044
Hoja: customer, Número de filas: 1392
Hoja: store, Número de filas: 2


In [ ]:
# Mostrar los primeros registros de la hoja 'Customer'
if 'customer' in spark_dfs:
    print("Mostrando los primeros registros de la tabla Customer:")
    spark_dfs['customer'].show(5)  # Cambia el número de filas a mostrar si es necesario
else:
    print("La hoja 'Customer' no está disponible.")

Mostrando los primeros registros de la tabla Customer:
+-----------+---------+-----------+----------+--------------------+-----------+-------+--------------------+--------------------+----------------+--------+
|customer_id| store_id| first_name| last_name|               email| address_id| active|         create_date|         last_update| customer_id_old| segment|
+-----------+---------+-----------+----------+--------------------+-----------+-------+--------------------+--------------------+----------------+--------+
|          1|        1|       MARY|     SMITH| MARY.SMITH@sakil...|          5|      1| 2006-02-14 22:04:36| 2006-02-15 04:57:20|            NULL|    NULL|
|          2|        1|   PATRICIA|   JOHNSON| PATRICIA.JOHNSON...|          6|      1| 2006-02-14 22:04:36| 2006-02-15 04:57:20|            NULL|    NULL|
|          3|        1|      LINDA|  WILLIAMS| LINDA.WILLIAMS@s...|          7|      1| 2006-02-14 22:04:36| 2006-02-15 04:57:20|            NULL|    NULL|
|        

In [ ]:
# Mostrar los primeros registros de la hoja 'Customer'
if 'film' in spark_dfs:
    print("Mostrando los primeros registros de la tabla Customer:")
    spark_dfs['film'].show(5)  # Cambia el número de filas a mostrar si es necesario
else:
    print("La hoja 'film' no está disponible.")

Mostrando los primeros registros de la tabla Customer:
+-------+-----------------+--------------------+-------------+------------+---------------------+----------------+------------+-------+-----------------+----------------+-------+-----------------+--------------------+
|film_id|            title|         description| release_year| language_id| original_language_id| rental_duration| rental_rate| length| replacement_cost| num_voted_users| rating| special_features|         last_update|
+-------+-----------------+--------------------+-------------+------------+---------------------+----------------+------------+-------+-----------------+----------------+-------+-----------------+--------------------+
|      1| ACADEMY DINOSAUR| A Epic Drama of ...|         2006|           1|                 NULL|               6|        0.99|     86|            20.99|           76750|     PG|   Deleted Scenes| 2020-01-25 14:40:46|
|      2|   ACE GOLDFINGER| A Astounding Epi...|         2006|           

# Experimento 3 codigo depurado aplciado

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_timestamp, trim
from pyspark.sql.types import IntegerType, FloatType
import pandas as pd

class FilmDataProcessorSpark:
    def __init__(self, file_path):
        self.file_path = file_path
        self.dataframes = {}
        self.spark = SparkSession.builder.master("local[*]").appName("ExcelToSpark")\
            .config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

    def load_data(self):
        sheet_names = ['film', 'inventory', 'rental', 'customer', 'store']
        for sheet in sheet_names:
            df_pandas = pd.read_excel(self.file_path, sheet_name=sheet)
            if not df_pandas.empty:
                df_spark = self.spark.createDataFrame(df_pandas)
                self.dataframes[sheet] = df_spark

    def clean_column_names(self, df):
        new_column_names = [col_name.strip().replace(' ', '_').replace('.', '_') for col_name in df.columns]
        renamed_columns = [f"`{old_name}` AS `{new_name}`" for old_name, new_name in zip(df.columns, new_column_names)]
        df = df.selectExpr(*renamed_columns)
        return df

    def clean_data(self):
        for sheet_name, df in self.dataframes.items():
            df = self.clean_column_names(df)
            if sheet_name == 'film':
                df = self.clean_film(df)
            elif sheet_name == 'inventory':
                df = self.clean_inventory(df)
            elif sheet_name == 'rental':
                df = self.clean_rental(df)
            elif sheet_name == 'customer':
                df = self.clean_customer(df)
            elif sheet_name == 'store':
                df = self.clean_store(df)
            self.dataframes[sheet_name] = df

    def clean_film(self, df):
        df = df.withColumn('film_id', regexp_replace(col('film_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('title', trim(col('title')))
        df = df.withColumn('description', trim(col('description')))
        df = df.withColumn('release_year', regexp_replace(col('release_year'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('language_id', regexp_replace(col('language_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('original_language_id', regexp_replace(col('original_language_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('rental_duration', regexp_replace(col('rental_duration'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('rental_rate', regexp_replace(col('rental_rate'), r'[^\d.]', '').cast(FloatType()))
        df = df.withColumn('length', regexp_replace(col('length'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('replacement_cost', regexp_replace(col('replacement_cost'), r'[^\d.]', '').cast(FloatType()))
        df = df.withColumn('rating', trim(col('rating')))
        df = df.withColumn('special_features', trim(col('special_features')))
        df = df.withColumn('last_update_film', to_timestamp(trim(col('last_update')), 'yyyy-MM-dd HH:mm:ss')).drop('last_update')
        return df

    def clean_inventory(self, df):
        df = df.withColumn('inventory_id', regexp_replace(col('inventory_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('film_id', regexp_replace(col('film_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('store_id', regexp_replace(col('store_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('last_update_inventory', to_timestamp(trim(col('last_update')), 'yyyy-MM-dd HH:mm:ss')).drop('last_update')
        return df

    def clean_rental(self, df):
        df = df.withColumn('rental_id', regexp_replace(col('rental_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('inventory_id', regexp_replace(col('inventory_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('customer_id', regexp_replace(col('customer_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('staff_id', regexp_replace(col('staff_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('rental_date', to_timestamp(trim(col('rental_date')), 'yyyy-MM-dd HH:mm:ss'))
        df = df.withColumn('return_date', to_timestamp(trim(col('return_date')), 'yyyy-MM-dd HH:mm:ss'))
        df = df.withColumn('last_update_rental', to_timestamp(trim(col('last_update')), 'yyyy-MM-dd HH:mm:ss')).drop('last_update')
        return df

    def clean_customer(self, df):
        df = df.withColumn('customer_id', regexp_replace(col('customer_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('store_id', regexp_replace(col('store_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('first_name', trim(col('first_name')))
        df = df.withColumn('last_name', trim(col('last_name')))
        df = df.withColumn('email', trim(col('email')))
        df = df.withColumn('address_id', regexp_replace(col('address_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('active', regexp_replace(col('active'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('create_date', to_timestamp(trim(col('create_date')), 'yyyy-MM-dd HH:mm:ss'))
        df = df.withColumn('last_update_customer', to_timestamp(trim(col('last_update')), 'yyyy-MM-dd HH:mm:ss')).drop('last_update')
        if 'customer_id_old' in df.columns:
            df = df.drop('customer_id_old')
        return df

    def clean_store(self, df):
        df = df.withColumn('store_id', regexp_replace(col('store_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('manager_staff_id', regexp_replace(col('manager_staff_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('address_id', regexp_replace(col('address_id'), r'[^0-9]', '').cast(IntegerType()))
        df = df.withColumn('last_update_store', to_timestamp(trim(col('last_update')), 'yyyy-MM-dd HH:mm:ss')).drop('last_update')
        return df

    def combine_data(self):
        # Unir inventory con film
        df_inventory_film = self.dataframes['inventory'].alias('inv').join(
            self.dataframes['film'].alias('film'),
            on='film_id',
            how='left'
        )

        # Unir inventory_film con store
        df_inventory_film_store = df_inventory_film.alias('inv_film').join(
            self.dataframes['store'].alias('store'),
            on=col('inv_film.store_id') == col('store.store_id'),
            how='left'
        ).select(
            [col('inv_film.' + c) for c in df_inventory_film.columns] +
            [col('store.' + c).alias(c + '_store') for c in self.dataframes['store'].columns if c != 'store_id']
        )

        # Unir rental con inventory_film_store
        df_rental_inventory_film_store = self.dataframes['rental'].alias('rental').join(
            df_inventory_film_store.alias('inv_film_store'),
            on='inventory_id',
            how='left'
        ).select(
            [col('rental.' + c) for c in self.dataframes['rental'].columns] +
            [col('inv_film_store.' + c) for c in df_inventory_film_store.columns if c != 'inventory_id']
        )

        # Unir rental_inventory_film_store con customer
        df_final = df_rental_inventory_film_store.alias('rental_inv_film_store').join(
            self.dataframes['customer'].alias('cust'),
            on='customer_id',
            how='left'
        ).select(
            [col('rental_inv_film_store.' + c) for c in df_rental_inventory_film_store.columns] +
            [col('cust.' + c).alias(c + '_customer') for c in self.dataframes['customer'].columns if c != 'customer_id']
        )

        return df_final

    def save_and_download(self, df, file_name):
        df.toPandas().to_excel(f'{file_name}.xlsx', index=False)
        try:
            from google.colab import files
            files.download(f'{file_name}.xlsx')
        except ImportError:
            print("El entorno no es Google Colab. El archivo se ha guardado localmente.")

# Uso de la clase FilmDataProcessorSpark
file_path = '/content/Films_2.xlsx'  # Ajustar la ruta del archivo
film_processor = FilmDataProcessorSpark(file_path)

# Cargar, limpiar, combinar y guardar los datos
film_processor.load_data()
film_processor.clean_data()
df_final = film_processor.combine_data()
df_final.printSchema()
df_final.show()
film_processor.save_and_download(df_final, 'Final_Combined_DataFrame')


root
 |-- rental_id: integer (nullable = true)
 |-- rental_date: timestamp (nullable = true)
 |-- inventory_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- return_date: timestamp (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- last_update_rental: timestamp (nullable = true)
 |-- film_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- last_update_inventory: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- language_id: integer (nullable = true)
 |-- original_language_id: integer (nullable = true)
 |-- rental_duration: integer (nullable = true)
 |-- rental_rate: float (nullable = true)
 |-- length: integer (nullable = true)
 |-- replacement_cost: float (nullable = true)
 |-- num_voted_users: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- special_features: string (nullable = true)
 |-- last_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>